In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os, sys
from shutil import copyfile

import io, tokenize, re, os

from itertools import combinations

from random import randint

from augment import find_all_files, flatten, intercalate_files
from DatasetUtils import py_cleaner, c_cleaner, add_to_list

working_dir = 'intercalate_working'

In [3]:
exts = ['java', 'py', 'js']

allowed_files = flatten([find_all_files(['old_data', 'new_data'], ext, 90) for ext in exts])
len(allowed_files)

1997

Out of all the files that was gotten, let's check which ones have labels

In [11]:
cols = ['quicksort', 'mergesort', 'selectionsort', 'insertionsort', 
        'bubblesort', 'linearsearch', 'binarysearch', 'linkedlist', 'hashmap']

final_file_list = []

for f in allowed_files:
    which_folder, label_folder, filename = f.split('/')
    #print(f)
    prefix = 'old' if which_folder == 'old_data' else 'new'
    
    df = pd.read_csv(f'csv_labels/{prefix}-{label_folder}.csv')
    loc = df.loc[df['Filename'] == filename]
    
    # If no such row exists, then let go of the file.
    if len(loc) == 0:
        continue
    else:
        haslabels = loc[cols].sum(axis=1).values.tolist()[0]
        # Only consider those with labels 
        if haslabels:
            # Then we add its column along with the filename
            row = [f'{prefix}_{label_folder}_{filename}', *flatten(loc[cols].values.tolist())]
            # Copy the file to our working directory.
            copyfile(f, f'{working_dir}/{prefix}_{label_folder}_{filename}')
            #print(row)
            final_file_list.append(row)
    
    

In [12]:
len(final_file_list)

1272

In [13]:
working_df = pd.DataFrame(final_file_list, columns=['Filename', *cols])

In [14]:
get_ext_files = lambda ext: list(filter(lambda x: f'.{ext}' in x, working_df['Filename']))

Augment data

In [15]:
final_df_list = []

for ext in exts:
    files = get_ext_files(ext)
    combs = combinations(files, 2)
    
    # Let's have a 5% chance of the file being intercalated.
    for comb in combs:
        intercalate = randint(0, 100) < 4
        if intercalate:
            
            rows = intercalate_files(working_df, ext, *list(map(lambda x: f'{working_dir}/{x}', list(comb))))
            if len(rows) > 0:
                for row in rows:
                    final_df_list.append(row)
print(f'Finished! Total augmented files: {len(final_df_list)}')

Finished! Total augmented files: 19143


In [16]:
final_df = pd.DataFrame(final_df_list, columns=['Filename', *cols])

In [17]:
final_df.to_csv('Intercalated.csv', index=False)